In [7]:
import numpy as np

In [26]:
Z_abc =  np.array([
[0.8667 + 2.0417j, 0.2955 + 0.9502j, 0.2907 + 0.7290j],
[0.2955 + 0.9502j, 0.8837 + 1.9852j, 0.2992 + 0.8023j],
[0.2907 + 0.7290j, 0.2992 + 0.8023j, 0.8741 + 2.0172j]
])

Y_abc =  np.array([
[ 10.7409j, -3.4777j, -1.3322j],
[ -3.4777j, 11.3208j, -2.2140j],
[ -1.3322j, -2.2140j, 10.2104j]
])*1.0e-6

U = np.eye(3)


In [27]:
a = U + 0.5 * Z_abc @ Y_abc
b = Z_abc
c = Y_abc + 0.25 * Y_abc @ Z_abc @ Y_abc
d = U + 0.5 * Z_abc @ Y_abc

In [28]:
c

array([[ -2.11511936e-11 +1.07408553e-05j,
          8.21431015e-12 -3.47768967e-06j,
          1.22583662e-13 -1.33219889e-06j],
       [  8.21431015e-12 -3.47768967e-06j,
         -2.35581400e-11 +1.13207537e-05j,
          3.37220770e-12 -2.21399501e-06j],
       [  1.22583662e-13 -1.33219889e-06j,
          3.37220770e-12 -2.21399501e-06j,
         -1.93260638e-11 +1.02103567e-05j]])